In [2]:
from sklearn.datasets import make_gaussian_quantiles
from sklearn.gaussian_process.kernels import RBF
from sklearn import svm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; 
import pandas as pd

In [3]:
df = pd.read_csv (r'.\winemag-data_first150k.csv')
print (df)

        Unnamed: 0 country                                        description  \
0                0      US  This tremendous 100% varietal wine hails from ...   
1                1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2                2      US  Mac Watson honors the memory of a wine once ma...   
3                3      US  This spent 20 months in 30% new French oak, an...   
4                4  France  This is the top wine from La Bégude, named aft...   
...            ...     ...                                                ...   
150925      150925   Italy  Many people feel Fiano represents southern Ita...   
150926      150926  France  Offers an intriguing nose with ginger, lime an...   
150927      150927   Italy  This classic example comes from a cru vineyard...   
150928      150928  France  A perfect salmon shade, with scents of peaches...   
150929      150929   Italy  More Pinot Grigios should taste like this. A r...   

                           